In [1]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'): 
        os.chdir('../')
set_root_path()
sys.path.append('python/')
sys.path = [p for p in sys.path if "unbiased-inverse-volume-rendering" not in p]
print(os.getcwd())
print(sys.executable)
os.environ.pop("PYTHONPATH")

c:\Users\mayuf\source\repos\practical-inverse-rendering-of-textured-and-translucent-appearance
c:\Users\mayuf\anaconda3\envs\practical-inverse-rendering\python.exe


'C:\\Users\\mayuf\\source\\repos\\unbiased-inverse-volume-rendering\\mitsuba3\\build\\Release\\python'

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import gridspec
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import mitsuba as mi
import drjit as dr
import numpy as np

from practical_reconstruction import figutils
from practical_reconstruction import scene_configuration
from practical_reconstruction import io_utils
from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

mi.set_variant("cuda_ad_rgb")

from core import integrators
from core import bsdfs
from core import textures
from core import emitters

integrators.register()
bsdfs.register()
textures.register()
emitters.register()

In [3]:
def load_modified_scene(scene_config, res=[400, 240]):
  os.makedirs(scene_config.result_folder,exist_ok=True)

  tmp_mitsuba_xml = io_utils.mitsuba_remote_to_local(scene_config,override=True)
  scene = mi.load_file(tmp_mitsuba_xml)
  params = mi.traverse(scene)

  params.update()
  return scene

In [4]:
def _update_light_position(params, camera_idx):
	# Update light position for the selected sensor
	light_positions = [
		["7.500000 0.000000 0.000000 0.300000 0.000000 7.471460 0.043578 2.949549 0.000000 -0.653668 0.498097 -35.366814 0.000000 0.000000 0.000000 1.000000"],
		["-7.500000 0.000001 -0.000000 0.300000 0.000001 7.500000 -0.000000 6.000000 0.000000 -0.000001 -0.500000 34.500000 0.000000 0.000000 0.000000 1.000000"],
		["7.328826 0.166872 0.105630 -7.094067 -0.000000 7.458747 -0.052370 9.665897 -1.593206 0.767620 0.485901 -34.513069 0.000000 0.000000 0.000000 1.000000"],
		["-7.328826 0.304331 -0.104258 7.598060 0.000001 7.361900 0.095509 -0.685621 1.593207 1.399935 -0.479592 33.071430 0.000000 0.000000 0.000000 1.000000"],
		["7.494684 0.133494 -0.016586 1.461014 0.000000 6.608728 0.236406 -10.548397 0.282341 -3.543571 0.440270 -31.318865 0.000000 0.000000 0.000000 1.000000"],
		["-7.494684 0.111304 0.017298 -0.910891 0.000001 6.892640 -0.197107 19.797482 -0.282341 -2.954509 -0.459184 31.642860 0.000000 0.000000 0.000000 1.000000"],
		["7.117022 -0.706185 -0.150544 10.838095 -0.000000 7.158140 -0.149236 16.446486 2.366009 2.124227 0.452841 -32.198891 0.000000 0.000000 0.000000 1.000000"],
		["-7.117022 -0.900310 0.145868 -9.910769 -0.000001 6.935801 0.190259 -7.318140 -2.366009 2.708157 -0.438776 30.214287 0.000000 0.000000 0.000000 1.000000"],
		["6.304254 -0.738532 0.266341 -18.343878 0.000000 7.375046 0.090889 -0.362262 -4.062806 -1.145979 0.413281 -29.429699 0.000000 0.000000 0.000000 1.000000"],
		["-6.304255 -0.387525 -0.269619 19.173311 -0.000000 7.465805 -0.047692 9.338423 4.062804 -0.601322 -0.418367 28.785717 0.000000 0.000000 0.000000 1.000000"],
		["6.311899 1.645186 -0.246786 17.575043 0.000000 6.853625 0.203063 -8.214446 4.050918 -2.563432 0.384528 -27.416950 0.000000 0.000000 0.000000 1.000000"],
		["-6.311899 1.316295 0.255406 -17.578449 0.000001 7.093016 -0.162469 17.372805 -4.050917 -2.050972 -0.397959 27.357143 0.000000 0.000000 0.000000 1.000000"],
		["7.316449 0.928876 0.090842 -6.058943 -0.000000 6.197108 -0.281629 25.714031 -1.649113 4.121049 0.403030 -28.712067 0.000000 0.000000 0.000000 1.000000"],
		["-7.316449 1.044102 -0.085099 6.256947 -0.000000 5.805343 0.316565 -16.159544 1.649113 4.632261 -0.377551 25.928574 0.000000 0.000000 0.000000 1.000000"],
		["6.835326 -2.120719 0.149531 -10.167147 0.000000 5.449726 0.343514 -18.046011 -3.086798 -4.696066 0.331117 -23.678185 0.000000 0.000000 0.000000 1.000000"],
		["-6.835326 -1.917162 -0.161284 11.589870 0.000001 5.878076 -0.310542 27.737961 3.086798 -4.245317 -0.357143 24.500000 0.000000 0.000000 0.000000 1.000000"],
		["5.221693 -0.906111 -0.353791 25.065395 -0.000000 7.393009 -0.084154 11.890759 5.383672 0.878849 0.343147 -24.520285 0.000000 0.000000 0.000000 1.000000"],
		["-5.221695 -1.365186 0.347180 -24.002619 0.000002 7.254859 0.126790 -2.875278 -5.383669 1.324115 -0.336735 23.071430 0.000000 0.000000 0.000000 1.000000"],
		["4.966681 1.186354 0.366210 -25.334677 0.000000 7.330979 -0.105551 13.388588 -5.619793 1.048480 0.323650 -23.155512 0.000000 0.000000 0.000000 1.000000"],
		["-4.966682 1.660596 -0.357923 25.354605 -0.000001 7.165092 0.147745 -4.342178 5.619792 1.467608 -0.316327 21.642857 0.000000 0.000000 0.000000 1.000000"],
		["6.495440 2.950367 -0.154267 11.098653 -0.000000 4.628530 0.393428 -21.539946 3.749567 -5.110973 0.267239 -19.206701 0.000000 0.000000 0.000000 1.000000"],
		["-6.495441 2.737462 0.170822 -11.657554 0.000001 5.125258 -0.365037 31.552601 -3.749565 -4.742155 -0.295918 20.214287 0.000000 0.000000 0.000000 1.000000"],
		["6.831079 -2.292678 -0.138724 10.010694 0.000000 5.040548 -0.370242 31.916969 3.096185 5.058310 0.306066 -21.924593 0.000000 0.000000 0.000000 1.000000"],
		["-6.831079 -2.465313 0.124875 -8.441247 -0.000000 4.537337 0.398121 -21.868475 -3.096186 5.439192 -0.275510 18.785715 0.000000 0.000000 0.000000 1.000000"],
		["4.241152 -3.143757 0.355149 -24.560425 -0.000000 6.459154 0.254116 -11.788103 -6.185679 -2.155488 0.243504 -17.545313 0.000000 0.000000 0.000000 1.000000"],
		["-4.241152 -2.667497 -0.372064 26.344473 -0.000001 6.766788 -0.215619 21.093304 6.185679 -1.828943 -0.255102 17.357143 0.000000 0.000000 0.000000 1.000000"],
		["3.585440 1.807922 -0.422301 29.861053 0.000000 7.212013 0.137224 -3.605716 6.587459 -0.984020 0.229851 -16.589569 0.000000 0.000000 0.000000 1.000000"],
		["-3.585439 1.248953 0.431199 -29.883900 -0.000000 7.363967 -0.094798 12.635845 -6.587460 -0.679784 -0.234694 15.928572 0.000000 0.000000 0.000000 1.000000"],
		["4.772033 3.920635 0.283677 -19.557381 -0.000000 5.515674 -0.338804 29.716280 -5.785993 3.233568 0.233964 -16.877499 0.000000 0.000000 0.000000 1.000000"],
		["-4.772033 4.276577 -0.259817 18.487190 0.000000 5.051755 0.369563 -19.869408 5.785992 3.527133 -0.214286 14.500000 0.000000 0.000000 0.000000 1.000000"],
		["7.172763 -2.072851 0.047364 -3.015504 -0.000000 2.431738 0.472989 -27.109219 -2.191226 -6.785274 0.155042 -11.352975 0.000000 0.000000 0.000000 1.000000"],
		["-7.172763 -2.003042 -0.059228 4.445971 -0.000001 3.040841 -0.457060 37.994175 2.191226 -6.556760 -0.193878 13.071429 0.000000 0.000000 0.000000 1.000000"],
		["3.266159 -3.596402 -0.380923 26.964638 -0.000000 6.347352 -0.266343 24.643972 6.751460 1.739834 0.184280 -13.399581 0.000000 0.000000 0.000000 1.000000"],
		["-3.266161 -4.080709 0.358577 -24.800425 0.000000 5.975000 0.302209 -15.154665 -6.751459 1.974130 -0.173469 11.642858 0.000000 0.000000 0.000000 1.000000"],
		["2.297698 -0.342003 0.475411 -32.978809 -0.000001 7.491390 0.023952 4.323370 -7.139369 -0.110069 0.153004 -11.210276 0.000000 0.000000 0.000000 1.000000"],
		["-2.297698 0.280822 -0.475590 33.591270 0.000000 7.494196 0.019667 4.623301 7.139369 0.090378 -0.153061 10.214287 0.000000 0.000000 0.000000 1.000000"],
		["2.714157 5.183684 -0.312785 22.194952 0.000001 5.032897 0.370705 -19.949331 6.991663 -2.012302 0.121423 -8.999601 0.000000 0.000000 0.000000 1.000000"],
		["-2.714157 4.755044 0.341714 -23.619980 -0.000000 5.498381 -0.340051 29.803574 -6.991663 -1.845904 -0.132653 8.785715 0.000000 0.000000 0.000000 1.000000"],
		["7.122634 2.323870 0.022872 -1.301074 0.000000 1.095397 -0.494638 40.624687 -2.349059 7.046257 0.069352 -5.354651 0.000000 0.000000 0.000000 1.000000"],
		["-7.353633 1.435309 -0.022506 1.875437 -0.000000 1.717186 0.486718 -28.070271 1.474477 7.158293 -0.112245 7.357143 0.000000 0.000000 0.000000 1.000000"],
		["2.099781 -5.825052 0.282132 -19.449261 0.000000 4.408278 0.404514 -22.315975 -7.200064 -1.698781 0.082279 -6.259549 0.000000 0.000000 0.000000 1.000000"],
		["-2.099782 -5.434044 -0.314904 22.343313 0.000000 4.920338 -0.377361 32.415260 7.200064 -1.584751 -0.091837 5.928572 0.000000 0.000000 0.000000 1.000000"],
		["1.080882 -0.334599 -0.494277 34.899403 -0.000000 7.492374 -0.022542 7.577931 7.421704 0.048731 0.071986 -5.538995 0.000000 0.000000 0.000000 1.000000"],
		["-1.080883 -0.979511 0.490452 -34.031654 -0.000001 7.434393 0.065990 1.380724 -7.421704 0.142653 -0.071429 4.500000 0.000000 0.000000 0.000000 1.000000"],
		["0.929983 3.678388 0.431301 -29.891064 -0.000001 6.519829 -0.247133 23.299314 -7.442119 0.459659 0.053896 -4.272741 0.000000 0.000000 0.000000 1.000000"],
		["-0.929984 4.228244 -0.408287 28.880077 -0.000001 6.171935 0.284075 -13.885275 7.442119 0.528372 -0.051020 3.071429 0.000000 0.000000 0.000000 1.000000"],
		["-0.882609 7.263825 -0.109704 7.979250 0.000000 1.657068 0.487643 -28.135040 7.447886 0.860797 -0.013000 0.410026 0.000000 0.000000 0.000000 1.000000"],
		["-2.018707 7.033868 0.109535 -7.367437 -0.000001 1.705981 -0.486893 40.082527 -7.223214 -1.965790 -0.030612 1.642857 0.000000 0.000000 0.000000 1.000000"],
		["0.307662 -6.150860 -0.285363 20.275440 -0.000000 4.284057 -0.410403 34.728199 7.493687 0.252531 0.011716 -1.320118 0.000000 0.000000 0.000000 1.000000"],
		["-0.307663 -6.500520 0.248539 -17.097710 0.000000 3.731221 0.433733 -24.361319 -7.493687 0.266888 -0.010204 0.214286 0.000000 0.000000 0.000000 1.000000"],
		["-0.160660 -2.892458 0.461196 -31.983715 0.000001 6.919527 0.192875 -7.501244 -7.498279 0.061975 -0.009882 0.191711 0.000000 0.000000 0.000000 1.000000"],
		["0.160657 -2.278515 -0.476247 33.637306 -0.000000 7.145347 -0.151936 16.635509 7.498279 0.048820 0.010204 -1.214286 0.000000 0.000000 0.000000 1.000000"],
		["-0.505623 3.712984 -0.433118 30.618282 -0.000000 6.511590 0.248097 -11.366772 7.482937 0.250887 -0.029266 1.548610 0.000000 0.000000 0.000000 1.000000"],
		["0.505624 3.132625 0.453044 -31.413086 0.000000 6.811156 -0.209318 20.652248 -7.482937 0.211672 0.030612 -2.642857 0.000000 0.000000 0.000000 1.000000"],
		["-1.925925 6.375299 0.229941 -15.795859 0.000001 3.568785 -0.439766 36.783653 -7.248504 -1.693914 -0.061095 3.776662 0.000000 0.000000 0.000000 1.000000"],
		["1.925925 6.651649 -0.192023 13.741608 -0.000001 2.980282 0.458829 -26.118034 7.248504 -1.767341 0.051020 -4.071429 0.000000 0.000000 0.000000 1.000000"],
		["-2.942106 -6.619630 0.129521 -8.766449 0.000000 2.112106 0.479764 -27.583469 -6.898841 2.823032 -0.055236 3.366510 0.000000 0.000000 0.000000 1.000000"],
		["2.942106 -6.425113 -0.167490 12.024331 -0.000000 2.731281 -0.465666 38.596626 6.898841 2.740077 0.071429 -5.500000 0.000000 0.000000 0.000000 1.000000"],
		["-1.549021 -2.774371 -0.452909 32.003601 0.000000 6.943335 -0.189034 19.232370 7.338292 -0.585635 -0.095603 6.192231 0.000000 0.000000 0.000000 1.000000"],
		["1.549021 -3.355918 0.435065 -30.154551 0.000001 6.669782 0.228658 -10.006060 -7.338292 -0.708391 0.091837 -6.928572 0.000000 0.000000 0.000000 1.000000"],
		["-1.738145 1.189327 0.479881 -33.291691 -0.000000 7.399677 -0.081508 11.705523 -7.295811 -0.283344 -0.114326 7.502840 0.000000 0.000000 0.000000 1.000000"],
		["1.738144 1.812166 -0.471145 33.280132 0.000001 7.264962 0.124192 -2.693458 7.295811 -0.431728 0.112245 -8.357143 0.000000 0.000000 0.000000 1.000000"],
		["-3.400968 5.741309 -0.228238 16.276623 -0.000001 3.841195 0.429445 -24.061165 6.684566 2.921060 -0.116122 7.628573 0.000000 0.000000 0.000000 1.000000"],
		["3.400968 5.421079 0.260728 -17.950975 -0.000000 4.388007 -0.405492 34.384460 -6.684566 2.758132 0.132653 -9.785715 0.000000 0.000000 0.000000 1.000000"],
		["-6.635746 -3.161510 -0.099370 7.255907 -0.000001 3.198366 -0.452256 37.657909 3.495266 -6.002109 -0.188654 12.705755 0.000000 0.000000 0.000000 1.000000"],
		["6.635746 -3.279389 0.080622 -5.343568 0.000000 2.594944 0.469119 -26.838306 -3.495266 -6.225904 0.153061 -11.214287 0.000000 0.000000 0.000000 1.000000"],
		["-3.178397 -4.371010 0.346680 -23.967592 0.000001 5.741240 0.321719 -16.520317 -6.793217 2.045101 -0.162204 10.854267 0.000000 0.000000 0.000000 1.000000"],
		["3.178394 -3.901153 -0.370758 26.253054 0.000001 6.139986 -0.287136 26.099504 6.793218 1.825262 0.173469 -12.642858 0.000000 0.000000 0.000000 1.000000"],
		["-2.916623 1.124330 -0.454504 32.115303 0.000001 7.400044 0.081359 0.304843 6.909653 0.474588 -0.191850 12.929502 0.000000 0.000000 0.000000 1.000000"],
		["2.916623 0.525862 0.459308 -31.851536 0.000000 7.478248 -0.038053 8.663689 -6.909653 0.221971 0.193878 -14.071429 0.000000 0.000000 0.000000 1.000000"],
		["-4.294805 3.661346 0.329304 -22.751261 0.000002 6.025269 -0.297741 26.841841 -6.148549 -2.557474 -0.230021 15.601463 0.000000 0.000000 0.000000 1.000000"],
		["4.294805 4.077924 -0.306777 21.774374 -0.000000 5.613094 0.331617 -17.213171 6.148549 -2.848458 0.214286 -15.500000 0.000000 0.000000 0.000000 1.000000"],
		["-7.499684 0.063355 -0.001792 0.425472 0.000000 2.929983 0.460267 -26.218662 0.068824 6.903708 -0.195324 13.172678 0.000000 0.000000 0.000000 1.000000"],
		["7.499684 0.060770 0.002154 0.149238 -0.000000 3.520557 -0.441491 36.904358 -0.068823 6.622084 0.234694 -16.928572 0.000000 0.000000 0.000000 1.000000"],
		["-4.963970 -3.189996 -0.308638 21.904667 -0.000001 6.175848 -0.283697 25.858795 5.622188 -2.816527 -0.272504 18.575289 0.000000 0.000000 0.000000 1.000000"],
		["4.963968 -3.581353 0.288929 -19.924999 0.000001 5.781459 0.318502 -16.295109 -5.622190 -3.162062 0.255102 -18.357143 0.000000 0.000000 0.000000 1.000000"],
		["-4.144962 -1.022679 0.411088 -28.476128 -0.000000 7.398932 0.081807 0.273499 -6.250543 0.678175 -0.272607 18.582497 0.000000 0.000000 0.000000 1.000000"],
		["4.144962 -0.481358 -0.415465 29.382576 0.000000 7.477727 -0.038505 8.695367 6.250544 0.319205 0.275510 -19.785715 0.000000 0.000000 0.000000 1.000000"],
		["-5.083177 3.096819 -0.304200 21.594027 -0.000001 6.205755 0.280781 -13.654707 5.514645 2.854524 -0.280400 19.127966 0.000000 0.000000 0.000000 1.000000"],
		["5.083175 2.687346 0.321037 -22.172558 0.000000 6.549215 -0.243655 23.055868 -5.514647 2.477086 0.295918 -21.214287 0.000000 0.000000 0.000000 1.000000"],
		["-7.326222 1.127744 0.076148 -5.030359 -0.000001 5.337012 -0.351292 30.590412 -1.605140 -5.147280 -0.347557 23.828972 0.000000 0.000000 0.000000 1.000000"],
		["7.326222 1.223004 -0.069306 5.151392 0.000000 4.857447 0.380965 -20.667545 1.605140 -5.582067 0.316326 -22.642857 0.000000 0.000000 0.000000 1.000000"],
		["-6.474771 -2.616503 0.182346 -12.464223 -0.000001 5.419583 0.345627 -18.193922 -3.785149 4.475716 -0.311916 21.334120 0.000000 0.000000 0.000000 1.000000"],
		["6.474770 -2.368159 -0.196855 14.079853 -0.000000 5.850810 -0.312822 27.897566 3.785150 4.050906 0.336735 -24.071428 0.000000 0.000000 0.000000 1.000000"],
		["-5.451219 -0.508029 -0.341735 24.221445 -0.000000 7.463435 -0.049312 9.451858 5.151137 -0.537624 -0.361643 24.814997 0.000000 0.000000 0.000000 1.000000"],
		["5.451218 -0.952858 0.337483 -23.323792 -0.000001 7.370567 0.092490 -0.474302 -5.151138 -1.008368 0.357143 -25.500000 0.000000 0.000000 0.000000 1.000000"],
		["-5.935410 0.985898 0.298507 -20.595474 -0.000000 7.324550 -0.107517 13.526174 -4.584857 -1.276312 -0.386437 26.550617 0.000000 0.000000 0.000000 1.000000"],
		["5.935410 1.372396 -0.291642 20.714970 -0.000000 7.156116 0.149666 -4.476626 4.584857 -1.776660 0.377551 -26.928574 0.000000 0.000000 0.000000 1.000000"],
		["-7.250820 1.221711 -0.098500 7.195035 -0.000000 5.779980 0.318621 -16.303457 1.917187 4.620525 -0.372530 25.577082 0.000000 0.000000 0.000000 1.000000"],
		["7.250820 1.088290 0.105224 -7.065701 0.000000 6.174529 -0.283825 25.867716 -1.917188 4.115922 0.397959 -28.357143 0.000000 0.000000 0.000000 1.000000"],
		["-7.281588 -0.772892 -0.108138 7.869672 -0.000001 6.770683 -0.215075 21.055218 1.796797 -3.132169 -0.438234 30.176382 0.000000 0.000000 0.000000 1.000000"],
		["7.281588 -0.911322 0.103236 -6.926512 0.000000 6.463745 0.253596 -11.751747 -1.796797 -3.693168 0.418367 -29.785717 0.000000 0.000000 0.000000 1.000000"],
		["-6.724941 -0.962456 0.211858 -14.530052 0.000001 7.178018 0.144930 -4.145110 -3.320417 1.949293 -0.429082 29.535753 0.000000 0.000000 0.000000 1.000000"],
		["6.724941 -0.681826 -0.216644 15.465076 -0.000000 7.340175 -0.102672 13.187013 3.320418 1.380923 0.438775 -31.214287 0.000000 0.000000 0.000000 1.000000"],
		["-6.935052 0.578879 -0.186428 13.349942 0.000001 7.344292 0.101355 -1.094834 2.855705 1.405801 -0.452738 31.191675 0.000000 0.000000 0.000000 1.000000"],
		["6.935052 0.332952 0.189082 -12.935728 0.000001 7.448849 -0.058296 10.080717 -2.855704 0.808572 0.459184 -32.642860 0.000000 0.000000 0.000000 1.000000"],
		["-7.418036 0.175292 0.072786 -4.795047 0.000001 7.405165 -0.079261 11.548292 -1.105778 -1.175927 -0.488282 33.679771 0.000000 0.000000 0.000000 1.000000"],
		["7.418036 0.269780 -0.071491 5.304364 -0.000000 7.273365 0.121987 -2.539059 1.105778 -1.809802 0.479592 -34.071430 0.000000 0.000000 0.000000 1.000000"],
		["-7.500000 0.000001 0.000000 0.300000 0.000001 7.471460 0.043578 2.949549 -0.000001 0.653668 -0.498097 34.366814 0.000000 0.000000 0.000000 1.000000"],
		["7.500000 0.000000 0.000000 0.300000 0.000000 7.500000 -0.000000 6.000000 0.000000 0.000001 0.500000 -35.500000 0.000000 0.000000 0.000000 1.000000"],
	]
	raw_string = light_positions[camera_idx][0]
	float_values = list(map(float, raw_string.strip().split()))
	matrix_4x4 = np.array(float_values).reshape((4, 4))
	params['arealight.to_world'] = mi.Transform4f(matrix_4x4)
	params.update()

def save_images(scene, params, dir, spp):
    print("start rendering")
    for i, sensor in enumerate(scene.sensors()):
					_update_light_position(params, i)
											
					integrator = mi.load_dict({
							'type': 'prb_path_volume',
							'max_sss_depth': -1,
							'max_path_depth': 256,
					})
					image = mi.render(scene, sensor=sensor, integrator=integrator, spp=spp)
					bitmap = mi.Bitmap(image).convert(
									pixel_format=mi.Bitmap.PixelFormat.RGB,
									component_format=mi.Struct.Type.UInt8,
									srgb_gamma=True
					)
					png_path = dir / f"ref_view_{i:03d}.png"
					bitmap.write(str(png_path))

					bitmap = mi.Bitmap(image).convert(
													pixel_format=mi.Bitmap.PixelFormat.RGB,
													component_format=mi.Struct.Type.Float32,
													srgb_gamma=False
					)
					exr_path = dir / f"ref_view_{i:03d}.exr"
					bitmap.write(str(exr_path))
					print(f"saving: {i}")

In [ ]:
# Display reference image
scene_name = 'kiwi_refine'
technique = 'mipmap_pyramid'
result_folder = f'results/{scene_name}/{technique}'
scene_folder = f'third_party/{scene_name}'

override_bindings = []
override_bindings.append(f"SceneConfig.result_folder='{result_folder}'")
override_bindings.append(f"SceneConfig.scene_folder='{scene_folder}'")
override_bindings.append(f'SceneConfig.base_learning_rate={0.1}')
scene_config = scene_configuration.SceneConfig.get_instance(
				f'{scene_name}/{technique}', override_bindings, sss_config=True
)

scene = load_modified_scene(scene_config,res=[800,480])
ref_spp=4096
opt_spp=128
params = mi.traverse(scene)

ref_dir=Path("third_party/kiwi_refine/references")
ref_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_refine_ref.xml'
ref_scene = mi.load_file(ref_scene_path)
ref_params = mi.traverse(ref_scene)
#save_images(ref_scene, ref_params, ref_dir, ref_spp)

# test_dir=Path("third_party/kiwi_refine/test")
# test_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_test.xml'
# test_scene = mi.load_file(test_scene_path)
# test_params = mi.traverse(test_scene)
#print(test_scene)
#save_images(test_scene, test_params, test_dir, ref_spp)

start rendering
saving: 0
saving: 1
saving: 2
saving: 3
saving: 4
saving: 5
saving: 6
saving: 7
saving: 8
saving: 9
saving: 10
saving: 11
saving: 12
saving: 13
saving: 14
saving: 15
saving: 16
saving: 17
saving: 18
saving: 19
saving: 20
saving: 21
saving: 22
saving: 23
saving: 24
saving: 25
saving: 26
saving: 27
saving: 28
saving: 29
saving: 30
saving: 31
saving: 32
saving: 33
saving: 34
saving: 35
saving: 36
saving: 37


In [ ]:
if technique == 'gradient_filtering':
    gin_config_name = f'{scene_name}/naive'
else:
    gin_config_name = f'{scene_name}/{technique}'

print(f'Next result location: {result_folder}')

optimization_cli.run_config(gin_config_name, override_bindings, sss_config=True)

Next result location: results/kiwi_refine/mipmap_pyramid
SceneConfig(approach=<Approach.MIPMAP_PYRAMID: 2>, scene_setup=<Setup.FULL_ON: 0>, random_lights=0, random_sensors=True, envmap_rotations=[0.0, 45.0], extra_render_resolutions=[], scene_name='kiwi_refine', scene_folder='third_party/kiwi_refine', result_folder='results/kiwi_refine/mipmap_pyramid', tmp_folder='tmp/kiwi_refine', n_resolutions=1, n_texture_scales=1, optimized_path_depth=20, rerender_spp=8192, samples_per_pixel_primal=256, samples_per_pixel_gradient=256, base_learning_rate=0.1, beta_1=0.9, use_sgd=False, mask_updates=False, use_mitsuba_reference=True, reference_scaling=1.0, reference_spp=1024, optimized_sensors_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], optimized_shapes=['slice'], per_material_learning_rates=defaultdict(<function SceneConfig.__post_init__.

Loading references for sensor elm__52: 100%|██████████| 1/1 [00:00<00:00, 994.62it/s]


Preparing optimization variables
Optimizing mat-kiwi's single_scattering_albedo from default value : [[0.5, 0.5, 0.5]]
Optimizing mat-kiwi's extinction_coefficient from default value : [[10, 10, 10]]
Optimizing mat-kiwi's hg_coefficient from default value : [[0, 0, 0]]
Optimizing mat-kiwi's clearcoat_gloss from default value : [0.5]
Optimizing mat-kiwi's roughness from default value : [0.5]
Optimizing mat-kiwi's normalmap from default value : [[0.5, 0.5, 1]]
Learning rate for (texture) mat-kiwi (single_scattering_albedo) is  0.1
Learning rate for (texture) mat-kiwi (extinction_coefficient) is  0.1
Learning rate for (texture) mat-kiwi (hg_coefficient) is  0.1
Learning rate for (texture) mat-kiwi (clearcoat_gloss) is  0.010000000000000002
Learning rate for (texture) mat-kiwi (roughness) is  0.010000000000000002
Learning rate for (texture) mat-kiwi (normalmap) is  0.010000000000000002
Starting optimization
Resizing initial value for mat-kiwi.nested_bsdf.single_scattering_albedo.data/mat-k

Rendering at resolution level 0 (width=436):   0%|          | 0/1024 [00:00<?, ?it/s]

0.00207509
0.00840627
12.4625


Rendering at resolution level 0 (width=436):   0%|          | 1/1024 [00:05<1:40:20,  5.88s/it]

13.0506
0.00743535
12.4614
0.00182155


Rendering at resolution level 0 (width=436):   0%|          | 2/1024 [00:08<1:10:20,  4.13s/it]

12.7581


In [ ]:
# Figure setup

def figure_grid_setup(
    fig_width,
    image_shape,
    image_crop_shape,
    inner_hspace=0.0,
    inner_wspace=0.0,
    outer_space=0.1,
):
  # Image aspect ratios
  h, w = image_shape
  h_crop, w_crop = image_crop_shape
  r = w / h
  r_crop = w_crop / h_crop

  # Spacing in the inner gridspec
  inner_wspace = inner_hspace
  # same vertical spacing as horizontal spacing
  inner_hspace = inner_wspace
  inner_rows = 2
  inner_cols = 3
  inner_height_ratios = [h_crop, h_crop]
  inner_width_ratios = [w_crop, w_crop, w_crop]
  inner_aspect = figutils.gridspec_aspect(
      n_rows=inner_rows,
      n_cols=inner_cols,
      w=inner_width_ratios,
      h=inner_height_ratios,
      wspace=inner_wspace,
      hspace=inner_hspace,
  )
  # Spacing in the main griddpec
  outer_rows = 2
  outer_cols = 1
  outer_wspace = 0
  outer_hspace = outer_space
  # If width is 1, we need the sum of the inverses for the height (single column)
  # If height is 1, we need the sum for the width (single row)
  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=[1],
      h=[1 / inner_aspect, 1 / r],
      wspace=outer_wspace,
      hspace=outer_hspace,
  )
  outer_aspect *= 0.98

  fig = plt.figure(1, figsize=(fig_width, fig_width / outer_aspect))

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1 / inner_aspect, 1 / r],
      width_ratios=[1],
  )

  inner_gs = gridspec.GridSpecFromSubplotSpec(
      inner_rows,
      inner_cols,
      subplot_spec=outer_gs[0],
      wspace=inner_wspace,
      hspace=inner_hspace,
      width_ratios=inner_width_ratios,
      height_ratios=inner_height_ratios,
  )
  return (
      fig,
      outer_gs,
      inner_gs,
      inner_rows,
      inner_cols,
  )

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "volume_gradients"

ref_crop_offset = (26, 22)
ref_crop_size = (739, 432)

grad_crop_offset = (18, 13)
grad_crop_size = (363, 215)

cropped_ref_image = figutils.crop_image(ref_image,ref_crop_offset,ref_crop_size)
cropped_forward_images = [figutils.crop_image(im,grad_crop_offset,grad_crop_size) for im in forward_images]
cropped_fd_images = [figutils.crop_image(im,grad_crop_offset,grad_crop_size) for im in fd_images]

titles = [
    "Albedo",
    "Extinction",
    "Phase function (HG)",
    r"\textsc{Dragon}",
]
row_titles = ["Our forward", "Finite diff."]

params_label = [
    r"$\rho=[0.2, 0.4, 0.95]$",
    r"$\sigma_t=[15,7,5]$",
    r"$g=[-0.5, -0.2, 0.1]$",
]

(
    fig,
    outer_gs,
    inner_gs,
    inner_rows,
    inner_cols,
) = figure_grid_setup(
    figutils.COLUMN_WIDTH,
    ref_image.shape[:2],
    cropped_forward_images[0].shape[:2],
    inner_hspace=0.02,
    inner_wspace=0.02,
    outer_space=0.0,
)

title_pad = 3
label_pad = 2
line_width = 0.75

strengths = [4.5, 4.5, 4.5]
# Grads
if True:
  for row in range(inner_rows):
    for col in range(inner_cols):
      ax = fig.add_subplot(inner_gs[row, col])
      figutils.disable_ticks(ax)
      if col == 0:
        ax.set_ylabel(row_titles[row], labelpad=label_pad)
      # Our
      if row == 0:
        ax.set_title(titles[col], pad=title_pad)
        im = ax.imshow(
            cropped_forward_images[col],
            cmap=cm.coolwarm,
            vmin=-forward_vlims[col] * strengths[col],
            vmax=forward_vlims[col] * strengths[col],
        )
      elif row == 1:
        im = ax.imshow(
            cropped_fd_images[col],
            cmap=cm.coolwarm,
            vmin=-forward_vlims[col] * strengths[col],
            vmax=forward_vlims[col] * strengths[col],
        )

# Reference
if True:
  ax = fig.add_subplot(outer_gs[1])
  im = ax.imshow(cropped_ref_image)
  ax.set_ylabel(titles[-1], labelpad=label_pad)
  # ax.set_title(r"\vspace{250px}" + r"\;\;\;\;\;\;\;\;\;".join(params_label),pad=0)
  ax.set_xlabel(r"\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;".join(params_label),labelpad=label_pad)
  figutils.disable_ticks(ax)

  ax = fig.add_subplot(outer_gs[0])
  ax.axis('off')
  # Create an inset axes
  inset = inset_axes(
      ax,  # Parent axes
      width="1%",  # Width of the inset in percentage of the parent axis
      height="99.19%",  # Height of the inset in percentage of the parent axis
      loc="lower left",  # Location inside the parent axis
      bbox_to_anchor=(1.005, 0.005, 1, 1),  # Bounding box adjustment
      bbox_transform=ax.transAxes,  # Use axes coordinates
      borderpad=0,
  )  # Padding

  # Create the colorbar
  cbar = fig.colorbar(
      cm.ScalarMappable(
          norm=Normalize(
              vmin=-np.max(forward_vlims), vmax=np.max(forward_vlims)
          ),
          cmap=cm.coolwarm,
      ),
      cax=inset,
      orientation="vertical",
  )
  cbar.ax.yaxis.set_label_position("left")  # Move label to the left
  # cbar.ax.yaxis.tick_left()  # Move ticks to the left side
  cbar.set_label("Parameter gradient", labelpad=-9)
  ticks = [cbar.vmin, cbar.vmax]
  cbar.set_ticks(ticks)
  # cbar.set_ticklabels([f"{x:.2f}" for x in ticks])
  cbar.set_ticklabels(["neg", "pos"])
  cbar.ax.tick_params(pad=1.8, length=0)

# fig.tight_layout()
figutils.force_post_crop_size(fig, figutils.COLUMN_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.COLUMN_WIDTH,
    backend=None,
)